In [22]:
import os
import torch
import yaml
from tqdm import tqdm
from ultralytics import YOLO

In [25]:
def find_and_set_config_path():
    current_path = os.getcwd()

    while not os.path.exists(os.path.join(current_path, 'config')):
        parent_path = os.path.dirname(current_path)

        # Sprawdź, czy osiągnęliśmy korzeń systemu plików
        if current_path == parent_path:
            print("Nie znaleziono folderu 'config' w żadnym z katalogów nadrzędnych.")
            return

        current_path = parent_path

    os.chdir(current_path)
    print(f'Znaleziono folder "config" w: {current_path}')
    
find_and_set_config_path()

Znaleziono folder "config" w: c:\Program Files (x86)\Common Files\projekt_nw


In [50]:
config_path = os.path.join('config', '1_per.yaml')

with open(config_path) as f:
    cfg = yaml.load(f, Loader=yaml.FullLoader)
    
photos_path = cfg['path_image']
model_file = cfg['model_detection']
output_file = cfg['path_data']
threshold = cfg['threshold_detection']

In [51]:
def count_people(photos_path, model_file, person_label=0., threshold=0.5, chunk_size=100):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = YOLO(model_file)
    model = model.to(device)
    
    photo_names = [photo_name for photo_name in os.listdir(photos_path)]
    photos_paths = [os.path.join(photos_path, photo_name) for photo_name in photo_names]
    
    counter_people = {}
    for i in tqdm(range(0, len(photos_paths), chunk_size)):
        chunk_paths = photos_paths[i:i+chunk_size]
        chunk_names = photo_names[i:i+chunk_size]

        results = model.predict(source=chunk_paths, conf=threshold, device=device, classes=person_label)
        results = {photo_name:len(result.boxes.cls) for photo_name, result in zip(chunk_names, results)}
        counter_people.update(results)
        break
    return counter_people

In [52]:
threshold_string = str(threshold).replace('.', ',')
output_file = os.path.join(output_file, f'photo_people_mappping_{threshold_string}.txt')

a = count_people(photos_path, model_file, chunk_size=100)

  0%|          | 0/1273 [00:00<?, ?it/s]


0: 640x640 (no detections), 1: 640x640 (no detections), 2: 640x640 (no detections), 3: 640x640 (no detections), 4: 640x640 (no detections), 5: 640x640 (no detections), 6: 640x640 (no detections), 7: 640x640 (no detections), 8: 640x640 (no detections), 9: 640x640 (no detections), 10: 640x640 (no detections), 11: 640x640 (no detections), 12: 640x640 (no detections), 13: 640x640 (no detections), 14: 640x640 (no detections), 15: 640x640 (no detections), 16: 640x640 (no detections), 17: 640x640 (no detections), 18: 640x640 (no detections), 19: 640x640 (no detections), 20: 640x640 (no detections), 21: 640x640 (no detections), 22: 640x640 (no detections), 23: 640x640 (no detections), 24: 640x640 (no detections), 25: 640x640 (no detections), 26: 640x640 (no detections), 27: 640x640 (no detections), 28: 640x640 (no detections), 29: 640x640 (no detections), 30: 640x640 (no detections), 31: 640x640 (no detections), 32: 640x640 (no detections), 33: 640x640 (no detections), 34: 640x640 (no detecti

  0%|          | 0/1273 [00:04<?, ?it/s]


In [53]:
with open(output_file, 'w') as file:
    file.write('\n'.join([f'{key}\t{value}' for key, value in a.items()]))